# Evolver Loop 2 Analysis

## Goals:
1. Analyze N=1 optimization potential (highest score contributor)
2. Understand what the optimal N=1 angle should be
3. Calculate potential gains from small N optimization

In [ ]:
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import matplotlib.pyplot as plt

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with rotation angle."""
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def get_bounding_box_side(poly):
    """Get the side length of the bounding square."""
    bounds = poly.bounds  # (minx, miny, maxx, maxy)
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

print("Tree dimensions at 0 degrees:")
tree_0 = get_tree_polygon(0, 0, 0)
bounds = tree_0.bounds
print(f"Width: {bounds[2] - bounds[0]:.6f}")
print(f"Height: {bounds[3] - bounds[1]:.6f}")
print(f"Bounding box side: {get_bounding_box_side(tree_0):.6f}")

In [ ]:
# Exhaustive search for optimal N=1 angle
angles = np.arange(0, 360, 0.01)  # 36000 angles
best_side = float('inf')
best_angle = 0
sides = []

for angle in angles:
    tree = get_tree_polygon(0, 0, angle)
    side = get_bounding_box_side(tree)
    sides.append(side)
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Optimal angle for N=1: {best_angle:.2f} degrees")
print(f"Optimal bounding box side: {best_side:.6f}")
print(f"Optimal N=1 score: {best_side**2:.6f}")

# Current baseline N=1 score
baseline_n1_score = 0.6612500000000001
baseline_n1_side = np.sqrt(baseline_n1_score)
print(f"\nBaseline N=1 side: {baseline_n1_side:.6f}")
print(f"Baseline N=1 score: {baseline_n1_score:.6f}")

print(f"\nPotential improvement: {baseline_n1_score - best_side**2:.6f}")

In [ ]:
# Plot the bounding box side vs angle
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(angles, sides)
plt.xlabel('Angle (degrees)')
plt.ylabel('Bounding box side')
plt.title('N=1: Bounding box side vs rotation angle')
plt.axhline(y=best_side, color='r', linestyle='--', label=f'Optimal: {best_side:.6f}')
plt.legend()

# Zoom in on optimal region
plt.subplot(1, 2, 2)
mask = (angles >= best_angle - 5) & (angles <= best_angle + 5)
plt.plot(angles[mask], np.array(sides)[mask])
plt.xlabel('Angle (degrees)')
plt.ylabel('Bounding box side')
plt.title(f'Zoomed: Optimal angle = {best_angle:.2f}°')
plt.axvline(x=best_angle, color='r', linestyle='--')
plt.tight_layout()
plt.savefig('/home/code/exploration/n1_angle_analysis.png', dpi=100)
plt.show()
print("Saved to /home/code/exploration/n1_angle_analysis.png")

In [ ]:
# Visualize the tree at optimal angle vs baseline angle
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# At 0 degrees
ax = axes[0]
tree = get_tree_polygon(0, 0, 0)
x, y = tree.exterior.xy
ax.fill(x, y, alpha=0.5, fc='green', ec='darkgreen')
ax.set_xlim(-0.6, 0.6)
ax.set_ylim(-0.6, 1.0)
ax.set_aspect('equal')
side = get_bounding_box_side(tree)
ax.set_title(f'0° - side={side:.4f}')
ax.axhline(y=tree.bounds[1], color='r', linestyle='--', alpha=0.5)
ax.axhline(y=tree.bounds[3], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[0], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[2], color='r', linestyle='--', alpha=0.5)

# At 45 degrees (baseline)
ax = axes[1]
tree = get_tree_polygon(0, 0, 45)
x, y = tree.exterior.xy
ax.fill(x, y, alpha=0.5, fc='green', ec='darkgreen')
ax.set_xlim(-0.8, 0.8)
ax.set_ylim(-0.8, 0.8)
ax.set_aspect('equal')
side = get_bounding_box_side(tree)
ax.set_title(f'45° (baseline) - side={side:.4f}')
ax.axhline(y=tree.bounds[1], color='r', linestyle='--', alpha=0.5)
ax.axhline(y=tree.bounds[3], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[0], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[2], color='r', linestyle='--', alpha=0.5)

# At optimal angle
ax = axes[2]
tree = get_tree_polygon(0, 0, best_angle)
x, y = tree.exterior.xy
ax.fill(x, y, alpha=0.5, fc='green', ec='darkgreen')
ax.set_xlim(-0.8, 0.8)
ax.set_ylim(-0.8, 0.8)
ax.set_aspect('equal')
side = get_bounding_box_side(tree)
ax.set_title(f'{best_angle:.2f}° (optimal) - side={side:.4f}')
ax.axhline(y=tree.bounds[1], color='r', linestyle='--', alpha=0.5)
ax.axhline(y=tree.bounds[3], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[0], color='r', linestyle='--', alpha=0.5)
ax.axvline(x=tree.bounds[2], color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('/home/code/exploration/n1_tree_comparison.png', dpi=100)
plt.show()

In [ ]:
# Calculate total potential improvement from optimizing N=1-10
import json

with open('/home/code/experiments/001_valid_baseline/metrics.json', 'r') as f:
    metrics = json.load(f)

baseline_per_n = metrics['per_n_scores']

print("Potential improvements from small N optimization:")
print("="*60)

total_improvement = 0
for n in range(1, 11):
    baseline_score = baseline_per_n[str(n)]
    baseline_side = np.sqrt(baseline_score * n)
    
    # For N=1, we know the optimal
    if n == 1:
        optimal_side = best_side
        optimal_score = best_side**2
    else:
        # For N>1, estimate potential (assume 5% improvement possible)
        optimal_score = baseline_score * 0.95
        optimal_side = np.sqrt(optimal_score * n)
    
    improvement = baseline_score - optimal_score
    total_improvement += improvement
    
    print(f"N={n:2d}: baseline={baseline_score:.6f}, optimal~={optimal_score:.6f}, improvement={improvement:.6f}")

print("="*60)
print(f"Total potential improvement from N=1-10: {total_improvement:.6f}")
print(f"\nCurrent total score: {metrics['cv_score']:.6f}")
print(f"Target score: 68.887744")
print(f"Gap to target: {metrics['cv_score'] - 68.887744:.6f}")
print(f"\nSmall N optimization could close {total_improvement / (metrics['cv_score'] - 68.887744) * 100:.1f}% of the gap")

In [ ]:
# Check what angle the baseline uses for N=1
import pandas as pd

df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')

# Get N=1 row
n1_row = df[df['id'] == '001_0'].iloc[0]

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

n1_x = parse_value(n1_row['x'])
n1_y = parse_value(n1_row['y'])
n1_deg = parse_value(n1_row['deg'])

print(f"Baseline N=1 configuration:")
print(f"  x = {n1_x}")
print(f"  y = {n1_y}")
print(f"  angle = {n1_deg}°")

# Calculate the bounding box for this configuration
tree = get_tree_polygon(n1_x, n1_y, n1_deg)
side = get_bounding_box_side(tree)
print(f"  bounding box side = {side:.6f}")
print(f"  score contribution = {side**2:.6f}")

print(f"\nOptimal angle found: {best_angle:.2f}°")
print(f"Optimal bounding box side: {best_side:.6f}")
print(f"Optimal score contribution: {best_side**2:.6f}")

print(f"\nImprovement from N=1 alone: {side**2 - best_side**2:.6f}")